In [5]:
#This file is the successor to ML1. Functions are a bit more modular
#draws functions from jurrefuncs.py
#will use this file to study the correlations of the multi cluster data when it becomes available
from jurrefuncs import *
import numpy as np


In [ ]:
import os
import nibabel as nib
from nibabel.testing import data_path
example_ni1 = os.path.join(data_path, 'example4d.nii.gz')
n1_img = nib.load(example_ni1)



from skimage import feature

edges1 = feature.canny(im)

In [6]:
#retrieve the data from several files and put it in datamatrix
from numpy import genfromtxt
dataFile = []
rangenr = 10
for i in range(1, rangenr):
    dataFile.append(genfromtxt("feature extraction\dataset\clustering\sum up of the feature\sumupCluster_"+ str(i) +"_train_org.csv", delimiter=','))
#dataMatrix = genfromtxt("feature extraction\dataset\clustering\sum up of the feature\sumupCluster_4_train_org.csv", delimiter=',')
featurelength = len(dataFile[0]) -1
#featurelength = 3

tempMatrix = np.array([dataFile[i][0:-1] for i in range(0, rangenr-1) ])
features = tempMatrix.reshape(-1, tempMatrix.shape[-1])
labels = dataFile[0][6]

#Also import the testdata

testdataFile = []
for i in range(1, rangenr):
    testdataFile.append(genfromtxt("feature extraction\dataset\clustering\sum up of the feature\sumupCluster_"+ str(i) +"_test_org.csv", delimiter=','))



testtempMatrix = np.array([testdataFile[i] for i in range(0, rangenr-1) ]) #does nothing
testfeatures = testtempMatrix.reshape(-1, testtempMatrix.shape[-1])



In [7]:
x_names = ['mean', 'variance' , 'standard', 'median', 'skewness', 'kurtosis']
y_name = 'Brain age'
#scatterPlotGrid(dataMatrix[0:6], dataMatrix[6], 3, 3, x_names, y_name) 

In [8]:
#Recursive feature selection with cross validation, seems to take a while sadly
#This is the one we are going to use in ML3
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge

#X = [[dataMatrix[i][j] for i in range(0, 6)] for j in range(0, len(dataMatrix[1]))   ]
#y = dataMatrix[6]

#X = [[features[i][j] for i in range(0, 5)] for j in range(0, len(features[0]))   ] 
X = [[features[i][j] for i in range(0, len(features) )] for j in range(0, len(features[0]))   ] 
y = labels

alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20, 100, 1000]
alpha = alpha_ridge[6]

ridgereg = Ridge(alpha=alpha,normalize=True)
selector = RFECV(ridgereg, step=1, cv=3)

# Fit the features to the response variable
# Put the best features into new df X_new
X_new = selector.fit_transform(X, y)
selectedBool = selector.support_ 
print(selectedBool)
y_pred = selector.predict(X)

rss = sum((y_pred-y)**2)
print (rss)
# 
testData = [[testfeatures[i][j] for i in range(0, len(features))] for j in range(0, len(testfeatures[0]))   ]
selector.transform(testData)
ans = selector.predict(testData)

import csv
file=open("submission_ridge.csv", "w")
c = csv.writer(file)
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans)):
    c.writerow([i+1 , ans[i]])
file.close()    

[ True False  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True False  True  True  True False
  True False  True  True  True False]
29583.8998323


In [9]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20, 100, 1000]
alpha = alpha_lasso[1]
print(alpha)

lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
selector = RFECV(lassoreg, step=1, cv=3)

# Fit the features to the response variable
# Put the best features into new df X_new
X_new = selector.fit_transform(X, y)
selectedBool = selector.support_ 
print(selectedBool)
y_pred = selector.predict(X)

rss = sum((y_pred-y)**2)
print (rss)
#           
testData = [[testfeatures[i][j] for i in range(0, len(features))] for j in range(0, len(testfeatures[0]))   ]
selector.transform(testData)
ans = selector.predict(testData)

import csv
file_lasso=open("submission_lasso.csv", "w")
c = csv.writer(file_lasso)
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans)):
    c.writerow([i+1 , ans[i]])
file_lasso.close() 
    

1e-10


C:\Users\mei\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\mei\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\mei\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\mei\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
C:\Users\mei\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to i

[ True False False False  True  True False False False False  True False
 False False False False  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False]
28957.4504564
